In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import *
from math import *
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.cross_validation import KFold, train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.grid_search import GridSearchCV

train = pd.read_csv('train.csv', delimiter=',')
train.describe()

hour = train['datetime'].apply(lambda s: s.split(' ')[1].split(':')[0])
hour = hour.apply(lambda s: int(s))
train.insert(1, 'hour', hour)

year = train['datetime'].apply(lambda s: s.split(' ')[0].split('-')[0])
year = year.apply(lambda s: int(s))
train.insert(1, 'year', year)

month = train['datetime'].apply(lambda s: s.split(' ')[0].split('-')[1])
month = month.apply(lambda s: int(s))
train.insert(2, 'month', month)

day = train['datetime'].apply(lambda s: s.split(' ')[0].split('-')[2])
day = day.apply(lambda s: int(s))
train.insert(3, 'day', day)

season = train['datetime'].apply(lambda s: s.split(' ')[0].split('-')[1])
season = season.apply(lambda s: 4 if int(s) == 1 or int(s) == 2 else (int(s) - 3) // 3 + 1)
train = train.drop('season', axis = 1)
train.insert(5, 'season', season)

#train.head(10)

In [4]:
test = pd.read_csv('test.csv', delimiter=',')
test.describe()
test.head(10)

hour = test['datetime'].apply(lambda s: s.split(' ')[1].split(':')[0])
hour = hour.apply(lambda s: int(s))
test.insert(1, 'hour', hour)
# test['hour']= test.index.hour

year = test['datetime'].apply(lambda s: s.split(' ')[0].split('-')[0])
year = year.apply(lambda s: int(s))
test.insert(1, 'year', year)

month = test['datetime'].apply(lambda s: s.split(' ')[0].split('-')[1])
month = month.apply(lambda s: int(s))
test.insert(2, 'month', month)

day = test['datetime'].apply(lambda s: s.split(' ')[0].split('-')[2])
day = day.apply(lambda s: int(s))
test.insert(3, 'day', day)

season = test['datetime'].apply(lambda s: s.split(' ')[0].split('-')[1])
season = season.apply(lambda s: 4 if int(s) == 1 or int(s) == 2 else (int(s) - 3) // 3 + 1)
test = test.drop('season', axis = 1)
test.insert(2, 'season', season)

#test.head(10)

In [5]:
#X = train[['hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']]
#X_test = test[['hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']]
X = train[['year', 'month', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']]
X_test = test[['year', 'month', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']]

In [6]:
maxDepth = 12
nEstimators = 150
subSample = 0.9

y = train['casual']
xgb_model = xgb.XGBRegressor(max_depth = maxDepth, n_estimators = nEstimators, subsample = subSample).fit(X,y)

predictions1 = xgb_model.predict(X)
#-----------------------------------
y = train['registered']
xgb_model = xgb.XGBRegressor(max_depth = maxDepth, n_estimators = nEstimators, subsample = subSample).fit(X,y)

predictions2 = xgb_model.predict(X)
#-----------------------------------
y = train['count']
predictions = predictions1 + predictions2
predictions = np.round(predictions)

print(min(abs(predictions - y)))
print(sum(abs(y - predictions)) / len(predictions))
print(max(abs(predictions - y)))

cnt = 0
for i in range(len(y)):
    cur = abs(predictions[i] - y[i]) / y[i]
    if (cur < 0.1):
        cnt += 1
print(cnt, len(predictions))

#train.insert(len(train.columns), 'my_count', predictions)
#train.head(10)

0.0
1.18142568437
18.0
10406 10886


In [7]:
y = train['casual']

xgb_model = xgb.XGBRegressor(max_depth = maxDepth, n_estimators = nEstimators, subsample = subSample).fit(X,y)
predictions1 = xgb_model.predict(X_test)
#--------------------------
y = train['registered']

xgb_model = xgb.XGBRegressor(max_depth = maxDepth, n_estimators = nEstimators, subsample = subSample).fit(X,y)
predictions2 = xgb_model.predict(X_test)
#--------------------------
predictions = predictions1 + predictions2
predictions = np.round(predictions)
predictions = [0 if predictions[i] < 0.01 else predictions[i] for i in range(predictions.shape[0])]

test.insert(len(test.columns), 'count', predictions)
test.head(20)

datetime  year  season  month  day  hour  holiday  workingday  \
0   2011-01-20 00:00:00  2011       4      1   20     0        0           1   
1   2011-01-20 01:00:00  2011       4      1   20     1        0           1   
2   2011-01-20 02:00:00  2011       4      1   20     2        0           1   
3   2011-01-20 03:00:00  2011       4      1   20     3        0           1   
4   2011-01-20 04:00:00  2011       4      1   20     4        0           1   
5   2011-01-20 05:00:00  2011       4      1   20     5        0           1   
6   2011-01-20 06:00:00  2011       4      1   20     6        0           1   
7   2011-01-20 07:00:00  2011       4      1   20     7        0           1   
8   2011-01-20 08:00:00  2011       4      1   20     8        0           1   
9   2011-01-20 09:00:00  2011       4      1   20     9        0           1   
10  2011-01-20 10:00:00  2011       4      1   20    10        0           1   
11  2011-01-20 11:00:00  2011       4      1   20    11        0           1   
12  2011-01-20 12:00:00  2011       4      1   20    12        0           1   
13  2011-01-20 13:00:00  2011       4      1   20    13        0           1   
14  2011-01-20 14:00:00  2011       4      1   20    14        0           1   
15  2011-01-20 15:00:00  2011       4      1   20    15        0           1   
16  2011-01-20 16:00:00  2011       4      1   20    16        0           1   
17  2011-01-20 17:00:00  2011       4      1   20    17        0           1   
18  2011-01-20 18:00:00  2011       4      1   20    18        0           1   
19  2011-01-20 19:00:00  2011       4      1   20    19        0           1   

    weather   temp   atemp  humidity  windspeed  count  
0         1  10.66  11.365        56    26.0027   15.0  
1         1  10.66  13.635        56     0.0000    6.0  
2         1  10.66  13.635        56     0.0000    5.0  
3         1  10.66  12.880        56    11.0014    3.0  
4         1  10.66  12.880        56    11.0014    3.0  
5         1   9.84  11.365        60    15.0013    9.0  
6         1   9.02  10.605        60    15.0013   32.0  
7         1   9.02  10.605        55    15.0013   84.0  
8         1   9.02  10.605        55    19.0012  198.0  
9         2   9.84  11.365        52    15.0013  124.0  
10        1  10.66  11.365        48    19.9995   60.0  
11        2  11.48  13.635        45    11.0014   56.0  
12        2  12.30  16.665        42     0.0000   75.0  
13        2  11.48  14.395        45     7.0015   68.0  
14        2  12.30  15.150        45     8.9981   78.0  
15        2  13.12  15.910        45    12.9980   82.0  
16        2  12.30  15.150        49     8.9981   94.0  
17        2  12.30  15.910        49     7.0015  191.0  
18        2  10.66  12.880        56    12.9980  173.0  
19        1  10.66  11.365        56    22.0028  105.0

In [8]:
newny = pd.DataFrame(test['datetime'])
newny.insert(1, 'count', predictions)
newny.head(20)

newny.to_csv('submission.csv', sep=',', header = True, index = False)